In [50]:
import pandas as pd
from bs4 import BeautifulSoup as bs
from splinter import Browser
import re

In [ ]:
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)

In [15]:

news_url = "https://mars.nasa.gov/news/?page=0&per_page=40&order=publish_date+desc%2Ccreated_at+desc&search=&category=19%2C165%2C184%2C204&blank_scope=Latest"
image_url = "https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars"

In [3]:
browser.visit(news_url)

In [6]:
html = browser.html
soup = bs(html, 'html.parser')

In [12]:
titles = soup.find_all('div', class_='content_title')
paras = soup.find_all('div', class_='article_teaser_body')
first_title = titles[1]
first_para = paras[0]

In [16]:
browser.visit(image_url)

In [19]:
html = browser.html
soup = bs(html, 'html.parser')

In [25]:
images = soup.find_all('article', class_='carousel_item')

In [46]:
image_link = "https://www.jpl.nasa.gov" + images[0].find('a')['data-fancybox-href']

In [47]:
image_link

'https://www.jpl.nasa.gov/spaceimages/images/mediumsize/PIA18640_ip.jpg'

In [48]:
twitter_url = "https://twitter.com/marswxreport?lang=en"
browser.visit(twitter_url)
html = browser.html
soup = bs(html, 'html.parser')

In [70]:
pattern = re.compile('InSight sol')

In [75]:
tweets = soup.find_all('span', class_='css-901oao')
tweets

[<span class="css-901oao css-16my406 css-bfa6kz r-1qd0xha r-ad9z0x r-bcqeeo r-qvutc0"></span>,
 <span class="css-901oao css-16my406 css-bfa6kz r-1qd0xha r-ad9z0x r-bcqeeo r-qvutc0"></span>,
 <span class="css-901oao css-16my406 css-bfa6kz r-1qd0xha r-ad9z0x r-bcqeeo r-qvutc0"></span>,
 <span class="css-901oao css-16my406 css-bfa6kz r-1qd0xha r-ad9z0x r-bcqeeo r-qvutc0"><span class="css-901oao css-16my406 r-1qd0xha r-ad9z0x r-bcqeeo r-qvutc0">Log in</span></span>,
 <span class="css-901oao css-16my406 r-1qd0xha r-ad9z0x r-bcqeeo r-qvutc0">Log in</span>,
 <span class="css-901oao css-16my406 css-bfa6kz r-1qd0xha r-ad9z0x r-bcqeeo r-qvutc0"><span class="css-901oao css-16my406 r-1qd0xha r-ad9z0x r-bcqeeo r-qvutc0">Sign up</span></span>,
 <span class="css-901oao css-16my406 r-1qd0xha r-ad9z0x r-bcqeeo r-qvutc0">Sign up</span>,
 <span class="css-901oao css-16my406 css-bfa6kz r-1qd0xha r-ad9z0x r-bcqeeo r-qvutc0"></span>,
 <span class="css-901oao css-16my406 r-1qd0xha r-ad9z0x r-bcqeeo r-qvutc0"

In [78]:
for tweet in tweets:
    test = tweet.find(pattern)
    print(test)

None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
